In [14]:
# %load bottle_db.py
#!/usr/bin/env python

import os
import dbm
import pickle

DB_PATH = os.getcwd()


class BottleBucket(object):
    def __init__(self, db_name):
        self.__dict__['db_name'] = db_name
        self.__dict__['db'] = dbm.open(
            os.path.join(DB_PATH, '%s' % db_name), 'c')
        self.__dict__['cached_memory'] = {}

    # 实现a['key'], a['key'] = value, del a['key']功能
    def __getitem__(self, key):
        if key not in self.cached_memory:
            self.cached_memory[key] = pickle.loads(self.db[key])
        return self.cached_memory[key]

    def __setitem__(self, key, value):
        self.cached_memory[key] = value

    def __delitem__(self, key):
        if key in self.cached_memory:
            del self.cached_memory[key]
        del self.db[key]

    # 实现a.key, a.key = value, del a.key功能
    def __getattr__(self, key):
        try:
            self[key]
        except KeyError:
            raise AttributeError(key)

    def __setattr__(self, key, value):
        self[key] = value

    def __delattr__(self, key):
        try:
            del self[key]
        except KeyError:
            raise AttributeError(key)

    def iter(self):
        return iter(self.ukeys())

    def __contains__(self, key):
        return key in self.ukeys()

    def __len__(self):
        return len(self.keys())

    def keys(self):
        return list(self.ukeys())

    def ukeys(self):
        return set(self.db.keys()) | set(self.cached_memory.keys())

    def update(self, other):
        self.cached_memory.update(other)

    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            if default:
                return default
            raise

    def clear(self):
        for key in self.db.keys():
            del self.db[key]
        self.cached_memory.clear()

    def save(self):
        self.close()
        self.__init__(self.db_name)

    def close(self):
        for key in self.cached_memory.keys():
            pvalue = pickle.dumps(self.cached_memory[key],
                                  pickle.HIGHEST_PROTOCOL)
            if key not in self.db or pvalue != self.db[key]:
                self.db[key] = pvalue
        self.cached_memory.clear()
        self.db.close()


In [15]:
db = BottleBucket('bottle')
db['a'] = 1
db.save()

In [26]:
db['b'] = 2
db.save()
del db.a
print(db.db.keys())

[b'b']


In [30]:
a = {'a': 1, 'b':2}
if 'a' in a:
    del a['a']
print(a)

{'b': 2}
